In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle
from sklearn.decomposition import PCA

In [ ]:
glove = {}
with open('embeddings/glove.txt', encoding = 'utf-8') as f: 
    for line in f: 
        temp = line.split()
        glove[temp[0]] = np.array([float(i) for i in temp[1:]])

In [ ]:
param = {}
with open('embeddings/paragm.txt', encoding = 'latin') as f: 
    for line in f: 
        temp = line.split(' ')
        key = temp[0]
        line = temp[1:]
        num = []
        for i in line:
            try:
                num.append(float(i))
            except:
                print(i)
                pass
        assert len(num) == 300
        param[key] = np.array(num)

In [ ]:
master = {}
for key in glove.keys():
    if key in param.keys(): 
        master[key] = 0.7*normalize(glove[key].reshape(1,-1))[0] + 0.3*normalize(param[key].reshape(1,-1))[0]

In [ ]:
with open('embeddings/master_v3.txt', mode = 'w', encoding = 'utf-8') as f:
    for key in master:
        f.write(key)
        for num in master[key]:
            f.write(' {}'.format(num))
        f.write('\n')

In [ ]:
test = pd.read_csv('data/cleaned_v4/test_cleaned_v4.csv')
train = pd.read_csv('data/cleaned_v4/train_cleaned_v4.csv')

In [ ]:
folds = []
for i in range(1,6): 
    folds.append(pd.read_csv('data/train_fold{}.csv'.format(i)))
train = pd.concat(folds)

In [ ]:
good = train[train.target == 0]
bad = train[train.target == 1]
g_val = good.sample(frac = 0.3)
b_val = bad.sample(frac = 0.3)
good = good.drop(g_val.index)
bad = bad.drop(b_val.index)
val = pd.concat(shuffle([g_val, b_val]))
train = pd.concat(shuffle([good, bad]))
train.to_csv('train.csv', index = False)
val.to_csv('val.csv',index = False)

In [ ]:
max_len = 70
def chop(sent): 
    sent = sent.split(' ')
    if len(sent) > max_len: 
        return ' '.join(sent)
    return ' '.join(sent)
test.question_text = test.question_text.apply(lambda x: chop(x))
train.question_text = train.question_text.apply(lambda x: chop(x))

In [ ]:
qid = test.qid
test = test.drop(['qid'], axis = 1)

In [ ]:
test.to_csv('test.csv')
qid.to_csv('qid.csv')

In [ ]:
good = train[train.target == 0].drop(['qid'],axis = 1)
bad = train[train.target == 1].drop(['qid'],axis = 1)

In [ ]:
def build_folds(good, bad, percent):
    folds = []
    for i in range(5):
        if i == 0:
            g = good.sample(frac= (percent*len(good)/len(good)))
            b = bad.sample(frac= (percent*len(bad)/len(bad)))
            amntg = len(g)
            amntb = len(b)
        else:
            g = good.sample(frac= (amntg/len(good)))
            b = bad.sample(frac= (amntb/len(bad)))
        good = good.drop(g.index)
        bad = bad.drop(b.index)
        folds.append(shuffle(pd.concat([g, b])))
        
    for f in folds:
        print(len(f))
    count = 1
    for fold in folds: 
        fold.to_csv('train_fold_5_{}.csv'.format(count), index = False)
        count+=1